In [27]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [52]:
%autoreload 2

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from data_prep import generate_cnn_data, split_preprocess_cnn_data


## Generate the CNN Data

In [55]:
df, features_df = generate_cnn_data(data_dir=os.path.join(os.getcwd(), '..', 'data', 'clean_data'), 
                         season='2020-21', 
                         position='GK', 
                         window_size=6,
                         verbose = True)
df.loc[0,'features']


======= Generating CNN Data for Season: 2020-21, Position: GK =======
Dropping Players with Avg. Playtime < 35...

Total players of type GK = 80.
60 players dropped due to low average playtime.
Generated windowed dataframe for CNN of shape: (632, 3).
Generated combined features dataframe for preprocessing of shape: (752, 21).

========== Done Generating CNN Data ==========



,position,team,opponent_team,minutes,goals_scored,assists,goals_conceded,clean_sheets,bps,yellow_cards,...,own_goals,saves,penalties_missed,penalties_saved,ict_index,influence,creativity,threat,was_home,total_points
0,GK,Sheffield Utd,Wolves,90,0,0,2,0,12,0,...,0,2,0,0,1.8,18.2,0.0,0.0,True,1
1,GK,Sheffield Utd,Aston Villa,90,0,0,1,0,10,0,...,0,1,0,0,1.1,11.2,0.0,0.0,False,2
2,GK,Sheffield Utd,Leeds,90,0,0,1,0,26,0,...,0,7,0,0,5.2,42.0,10.0,0.0,True,4
3,GK,Sheffield Utd,Arsenal,90,0,0,2,0,13,0,...,0,3,0,0,2.1,18.6,0.0,2.0,False,2
4,GK,Sheffield Utd,Fulham,90,0,0,1,0,18,0,...,0,5,0,0,2.8,27.8,0.0,0.0,True,3
5,GK,Sheffield Utd,Liverpool,90,0,0,2,0,14,0,...,0,3,0,0,2.2,21.8,0.0,0.0,False,2


In [56]:
df.head(5)

,name,features,target
0,Aaron Ramsdale,position team opponent_team minut...,4
1,Aaron Ramsdale,position team opponent_team minut...,2
2,Aaron Ramsdale,position team opponent_team minut...,2
3,Aaron Ramsdale,position team opponent_team minut...,3
4,Aaron Ramsdale,position team opponent_team minut...,1


In [57]:
df.tail(5)

,name,features,target
627,Kasper Schmeichel,position team opponent_team minute...,2
628,Kasper Schmeichel,position team opponent_team minute...,0
629,Kasper Schmeichel,position team opponent_team minute...,2
630,Kasper Schmeichel,position team opponent_team minute...,2
631,Kasper Schmeichel,position team opponent_team minute...,-2


In [58]:
df.loc[0,'features'].shape

(6, 21)

In [59]:
df.loc[0,'features']

,position,team,opponent_team,minutes,goals_scored,assists,goals_conceded,clean_sheets,bps,yellow_cards,...,own_goals,saves,penalties_missed,penalties_saved,ict_index,influence,creativity,threat,was_home,total_points
0,GK,Sheffield Utd,Wolves,90,0,0,2,0,12,0,...,0,2,0,0,1.8,18.2,0.0,0.0,True,1
1,GK,Sheffield Utd,Aston Villa,90,0,0,1,0,10,0,...,0,1,0,0,1.1,11.2,0.0,0.0,False,2
2,GK,Sheffield Utd,Leeds,90,0,0,1,0,26,0,...,0,7,0,0,5.2,42.0,10.0,0.0,True,4
3,GK,Sheffield Utd,Arsenal,90,0,0,2,0,13,0,...,0,3,0,0,2.1,18.6,0.0,2.0,False,2
4,GK,Sheffield Utd,Fulham,90,0,0,1,0,18,0,...,0,5,0,0,2.8,27.8,0.0,0.0,True,3
5,GK,Sheffield Utd,Liverpool,90,0,0,2,0,14,0,...,0,3,0,0,2.2,21.8,0.0,0.0,False,2


In [60]:
features_df

,position,team,opponent_team,minutes,goals_scored,assists,goals_conceded,clean_sheets,bps,yellow_cards,...,own_goals,saves,penalties_missed,penalties_saved,ict_index,influence,creativity,threat,was_home,total_points
0,GK,Sheffield Utd,Wolves,90,0,0,2,0,12,0,...,0,2,0,0,1.8,18.2,0.0,0.0,True,1
1,GK,Sheffield Utd,Aston Villa,90,0,0,1,0,10,0,...,0,1,0,0,1.1,11.2,0.0,0.0,False,2
2,GK,Sheffield Utd,Leeds,90,0,0,1,0,26,0,...,0,7,0,0,5.2,42.0,10.0,0.0,True,4
3,GK,Sheffield Utd,Arsenal,90,0,0,2,0,13,0,...,0,3,0,0,2.1,18.6,0.0,2.0,False,2
4,GK,Sheffield Utd,Fulham,90,0,0,1,0,18,0,...,0,5,0,0,2.8,27.8,0.0,0.0,True,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,GK,Leicester,Southampton,90,0,0,1,0,11,0,...,0,2,0,0,1.4,14.2,0.0,0.0,False,2
34,GK,Leicester,Newcastle,90,0,0,4,0,12,0,...,0,2,0,0,1.4,14.0,0.0,0.0,True,0
35,GK,Leicester,Man Utd,90,0,0,1,0,7,0,...,0,0,0,0,0.3,2.6,0.0,0.0,False,2
36,GK,Leicester,Chelsea,90,0,0,2,0,17,0,...,0,4,0,0,3.7,36.6,0.0,0.0,False,2


In [61]:
features_df.describe()

,minutes,goals_scored,assists,goals_conceded,clean_sheets,bps,yellow_cards,red_cards,own_goals,saves,penalties_missed,penalties_saved,ict_index,influence,creativity,threat,total_points
count,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000,752.00000,752.0,752.000000,752.000000,752.000000,752.000000,752.000000,752.000000
mean,80.575798,0.001330,0.002660,1.188830,0.271277,16.813830,0.033245,0.001330,0.003989,2.69016,0.0,0.014628,2.103324,20.852394,0.146543,0.038564,3.441489
std,27.395259,0.036466,0.051537,1.278307,0.444914,9.168633,0.179394,0.036466,0.063077,2.04308,0.0,0.120137,1.486558,14.663090,1.201348,0.644429,3.055942
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000
25%,90.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,1.00000,0.0,0.000000,1.000000,10.400000,0.000000,0.000000,1.000000
50%,90.000000,0.000000,0.000000,1.000000,0.000000,16.000000,0.000000,0.000000,0.000000,2.00000,0.0,0.000000,2.000000,19.600000,0.000000,0.000000,2.000000
75%,90.000000,0.000000,0.000000,2.000000,1.000000,23.000000,0.000000,0.000000,0.000000,4.00000,0.0,0.000000,3.025000,30.400000,0.000000,0.000000,6.000000
max,90.000000,1.000000,1.000000,9.000000,1.000000,45.000000,1.000000,1.000000,1.000000,11.00000,0.0,1.000000,7.600000,75.600000,10.000000,17.000000,15.000000


## Split & Preprocess CNN Data

In [63]:
X_train, y_train, X_val, y_val, X_test, y_test = split_preprocess_cnn_data(df, 
                                                            features_df, 
                                                            verbose=True)

# Print shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

========== Splitting CNN Data ==========

Shape of windowed_df: (632, 3)
Shape of a given window: (6, 21)
========== Preprocessing CNN Data ==========

Mean of Standard Scaler:
[8.05757979e+01 1.32978723e-03 2.65957447e-03 1.18882979e+00
 2.71276596e-01 1.68138298e+01 3.32446809e-02 1.32978723e-03
 3.98936170e-03 2.69015957e+00 0.00000000e+00 1.46276596e-02
 2.10332447e+00 2.08523936e+01 1.46542553e-01 3.85638298e-02
 3.44148936e+00]

Standard Deviation of Standard Scaler:
[27.37703843  0.03644199  0.05150244  1.27745705  0.44461849  9.16253474
  0.17927485  0.03644199  0.06303528  2.04172147  1.          0.12005703
  1.48556963 14.65333705  1.20054897  0.64400018  3.05390958]
Transforming features using StandardScaler + OHE Pipeline.
========== Done Preprocessing CNN Data ==========

========== Done Splitting CNN Data ==========

X_train shape: (384, 6, 59)
y_train shape: (384,)
X_val shape: (58, 6, 59)
y_val shape: (58,)
X_test shape: (190, 6, 59)
y_test shape: (190,)


In [ ]:
X_train[0:2]

array([[['GK', 'Sheffield Utd', 'Wolves', 90, 0, 0, 2, 0, 12, 0, 0, 0,
         2, 0, 0, 1.8, 18.2, 0.0, 0.0, True, 1],
        ['GK', 'Sheffield Utd', 'Aston Villa', 90, 0, 0, 1, 0, 10, 0, 0,
         0, 1, 0, 0, 1.1, 11.2, 0.0, 0.0, False, 2],
        ['GK', 'Sheffield Utd', 'Leeds', 90, 0, 0, 1, 0, 26, 0, 0, 0, 7,
         0, 0, 5.2, 42.0, 10.0, 0.0, True, 4],
        ['GK', 'Sheffield Utd', 'Arsenal', 90, 0, 0, 2, 0, 13, 0, 0, 0,
         3, 0, 0, 2.1, 18.6, 0.0, 2.0, False, 2],
        ['GK', 'Sheffield Utd', 'Fulham', 90, 0, 0, 1, 0, 18, 0, 0, 0,
         5, 0, 0, 2.8, 27.8, 0.0, 0.0, True, 3],
        ['GK', 'Sheffield Utd', 'Liverpool', 90, 0, 0, 2, 0, 14, 0, 0,
         0, 3, 0, 0, 2.2, 21.8, 0.0, 0.0, False, 2]],

       [['GK', 'Sheffield Utd', 'Aston Villa', 90, 0, 0, 1, 0, 10, 0, 0,
         0, 1, 0, 0, 1.1, 11.2, 0.0, 0.0, False, 2],
        ['GK', 'Sheffield Utd', 'Leeds', 90, 0, 0, 1, 0, 26, 0, 0, 0, 7,
         0, 0, 5.2, 42.0, 10.0, 0.0, True, 4],
        ['GK', 'Shef